# Demo Emotion Recognition: audio + video

## Libraries and parameters

In [1]:
# Utilities
import os
import subprocess
import numpy as np
import keras

# Audio and video manipulation
import moviepy.editor as mp
import cv2
import librosa
from joblib import load
import tkinter as tk

In [2]:
# Labels dictionary
emotions_tras = {1:1, 2:4, 3:5, 4:0, 5:3, 6:2, 7:6}
emotions = {0:'angry', 1:'calm', 2:'disgust', 3:'fear', 4:'happy', 5:'sad', 6:'surprise'}

# Paths
dataset_path = "./Examples/"
haar_path = './../Other/haarcascade_frontalface_default.xml'
parameters_path = './../Datasets/RAVDESS_audio/std_scaler.bin'
models_video_path = "./../Models/Video Stream/"
models_audio_path = "./../Models/Audio Stream/"
vlc_path = "C:/Program Files (x86)/VideoLAN/VLC/vlc.exe" # to play the selected video (insert your own path to vlc.exe)

# Audio video parameters
height_targ = 112
width_targ = 112
sr = 48000

## Select Clip

In [3]:
root= tk.Tk()

canvas1 = tk.Canvas(root, width=400, height=300, relief='raised')
canvas1.pack()

label1 = tk.Label(root, text='Select clip to analize')
label1.config(font=('helvetica', 16))
canvas1.create_window(200, 25, window=label1)

label2 = tk.Label(root, text='Number from 0 to 6:')
label2.config(font=('helvetica', 11))
canvas1.create_window(200, 100, window=label2)

def display_text():
   global example
   example = int(example.get())
   root.destroy

example = tk.Entry(root)
example.pack()
canvas1.create_window(200, 140, window=example)

    
button1 = tk.Button(text='Select', command=lambda: [display_text(), root.destroy()], font=('helvetica', 12, 'bold'))
canvas1.create_window(200, 180, window=button1)

root.mainloop()

In [4]:
fn = os.listdir(dataset_path)
filename = dataset_path + fn[example]
label = emotions_tras[int(fn[example].split('-')[2]) - 1] # trasposition of the emotions

In [5]:
player = subprocess.call([vlc_path, filename, '--play-and-exit'])

## Data preparation

### Video

In [6]:
cap = cv2.VideoCapture(filename)
haar_cascade = cv2.CascadeClassifier(haar_path)
frames = []
count = 0
skip = 3

# Loop through all frames
while True:
    # Capture frame
    ret, frame = cap.read()
    if (count % skip == 0 and count > 20):
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # detect and crop face
        faces = haar_cascade.detectMultiScale(frame, scaleFactor=1.12, minNeighbors=9)
        if len(faces) != 1:
            continue
        for (x, y, w, h) in faces:
            face = frame[y:y + h, x:x + w]

        face = cv2.resize(face, (height_targ+10, width_targ+10))
        face = face[5:-5, 5:-5]
        face = face/255.
        frames.append(face)
    count += 1

frames = np.array(frames)
num_frames = len(frames)
labels = [label] * num_frames
print('shape frames:', frames.shape)

shape frames: (38, 112, 112)


### Audio

In [7]:
audiofile = mp.AudioFileClip(filename).set_fps(sr)
audio = audiofile.to_soundarray()
audio = audio[int(sr/2):int(sr/2 + sr*3)]
audio = np.array([elem[0] for elem in audio])

In [8]:
mel = librosa.power_to_db(librosa.feature.melspectrogram(y=audio, sr = 48000, n_fft = 1024, n_mels = 128, fmin = 50, fmax = 24000)) 

scaler = load(parameters_path)
mel = scaler.transform(mel)

mel = np.expand_dims(mel, axis = 2)
mel = np.expand_dims(mel, axis = 0)
mel.shape

(1, 128, 282, 1)

## Load models

### Video

In [9]:
models_list = os.listdir(models_video_path)

acc = [float(model.split('[')[1].split(']')[0]) for model in models_list]
idx = acc.index(max(acc))                                                       # index of best model

model_video = keras.models.load_model(models_video_path + models_list[idx])
# model_video.summary()

### Audio

In [10]:
models_list = os.listdir(models_audio_path)
model_audio = keras.models.load_model(models_audio_path + models_list[0])
# model_audio.summary()

## Predictions

### Video

In [11]:

pred = model_video.predict(frames)
pred_video = np.mean(pred, axis=0)
pred_video

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 226ms/step


array([1.0827730e-01, 5.0532678e-04, 7.8479534e-01, 1.5410375e-02,
       4.8365715e-04, 8.4291272e-02, 6.2367585e-03], dtype=float32)

### Audio

In [12]:
pred = model_audio.predict(mel)
pred_audio = np.mean(pred, axis=0)
pred_audio

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step


array([1.2039708e-01, 7.1477855e-04, 5.7058519e-01, 2.6877111e-01,
       3.8588814e-02, 6.0333859e-04, 3.3959892e-04], dtype=float32)

### Global

In [13]:
pred_global = pred_video + pred_audio # mean

In [14]:
print('Video prediction:\t', emotions[pred_video.argmax()])
print('Audio prediction:\t', emotions[pred_audio.argmax()])
print('Global prediction:\t', emotions[pred_global.argmax()])

print('Ground truth:\t\t', emotions[label])

Video prediction:	 disgust
Audio prediction:	 disgust
Global prediction:	 disgust
Ground truth:		 disgust


In [15]:
# Print Results
root = tk.Tk()

# root window title and dimension
root.title("Results predictions")

canvas1 = tk.Canvas(root, width=400, height=300, relief='raised')
canvas1.pack()

label1 = tk.Label(root, text=f'Video prediction:\t{emotions[pred_video.argmax()]}')
label2 = tk.Label(root, text=f'Audio prediction:\t{emotions[pred_audio.argmax()]}')
label3 = tk.Label(root, text=f'Global prediction:\t{emotions[pred_global.argmax()]}')
label4 = tk.Label(root, text=f'Ground truth:\t{emotions[label]}')

label1.config(font=('helvetica', 14))
label2.config(font=('helvetica', 14))
label3.config(font=('helvetica', 14))
label4.config(font=('helvetica', 14), fg='gray')

canvas1.create_window(20, 25, window=label1, anchor='w')
canvas1.create_window(20, 50, window=label2, anchor='w')
canvas1.create_window(20, 75, window=label3, anchor='w')
canvas1.create_window(20, 120, window=label4, anchor='w')

button1 = tk.Button(text='Close', command=lambda: root.destroy(), font=('helvetica', 12, 'bold'))
canvas1.create_window(200, 200, window=button1)

root.mainloop()